In [9]:
import os
import numpy as np
from osgeo import gdal

In [ ]:
# defining the dataset
os.getcwd() 
folder1 = (r'F:\final thesis\thesis_data\thesis_data\shivanshi\single_pixel_data\surface_water_1_degree\sw_1_degree_oc\anomaly')

In [ ]:
# Define a list of years to process
years = ['2004', '2005', '2006', '2007', '2008', '2009']

In [ ]:
# getting all the tiff files in all the folders. For that generating a loop that joins folder1 and year_folder
for year_folder in years:
    folder = os.path.join(folder1, year_folder)
    tif_files = [f for f in os.listdir(folder) if f.endswith('.tif')]
    print(year_folder, tif_files)

In [ ]:
# printing the all the tif files to check its coming correct or not
print(tif_files[0])

In [ ]:
# Create an empty array to store the data from all the TIFF files
data = []

In [ ]:
# Loop through each year subfolder and collect all the matching TIFF files
for year in years:
    folder = os.path.join(folder1, year)
    
    #list of all tif files in the folder for converting all of them into array for mean calculation
    tif_files = [f for f in os.listdir(folder) if f.endswith('.tif')]
    
    # Loop through each TIFF file in the folder and read its data
    for tif_file in tif_files:
        tiff_path = os.path.join(folder, tif_file)
        tiff_dataset = gdal.Open(tiff_path)
        tiff_band = tiff_dataset.GetRasterBand(1)
        tiff_data = tiff_band.ReadAsArray()
        data.append(tiff_data)

In [ ]:
# Print the shape of each array in the data list
for tiff_data in data:
    print(data)

In [ ]:
# Calculate the long term average
long_term_average = np.mean(data, axis=0)

In [ ]:
print(long_term_average)

In [ ]:
# Define the output file path
output_file = r"F:\final thesis\thesis_data\thesis_data\shivanshi\single_pixel_data\surface_water_1_degree\sw_1_degree_oc\anomaly\baseline_mean.tif"

# Define the geotransform and projection of the output file
geotransform = tiff_dataset.GetGeoTransform()
projection = tiff_dataset.GetProjection()

# Create a new TIFF file with the same dimensions as the input data
driver = gdal.GetDriverByName("GTiff")
rows, cols = tiff_data.shape
out_dataset = driver.Create(output_file, cols, rows, 1, gdal.GDT_Float32)

# Write the long term average to the output file
out_band = out_dataset.GetRasterBand(1)
out_band.WriteArray(long_term_average)

# Set the geotransform and projection of the output file
out_dataset.SetGeoTransform(geotransform)
out_dataset.SetProjection(projection)

# Close the output file
out_dataset = None

In [3]:
ds1 = gdal.Open(r'F:\final thesis\thesis_data\thesis_data\shivanshi\single_pixel_data\surface_water_0.25_deree\0.25_sw_oc\0.25_sw_converted_oc\anomaly\baseline_mean.tif')
print(ds1)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001858686D480> >


In [4]:
array1 = ds1.GetRasterBand(1).ReadAsArray()
# print(array1)

In [5]:
# Dimensions
cols = ds1.RasterXSize
rows = ds1.RasterYSize
bands = ds1.RasterCount
print(rows, cols, bands)

4 4 1


In [6]:
ds2 = gdal.Open(r'F:\final thesis\thesis_data\thesis_data\shivanshi\single_pixel_data\surface_water_0.25_deree\0.25_sw_oc\0.25_sw_converted_oc\December_sw_2002.tif')

In [7]:
array2 = ds2.GetRasterBand(1).ReadAsArray()
# print(array2)

In [8]:
anomaly = array2 - array1
print(anomaly)

[[  0.         0.         0.         0.      ]
 [  0.         0.        32.044205 107.032776]
 [  0.         0.         0.         0.      ]
 [  0.         0.         0.         0.      ]]


In [ ]:
driver = gdal.GetDriverByName("GTiff")
out_ds = driver.Create(r"F:\final thesis\thesis_data\thesis_data\shivanshi\single_pixel_data\surface_water_1_degree\sw_1_degree_oc\sw_1_degree_oc_converted\anomaly_converted\December_sw_2002.tif", cols, rows, 1, gdal.GDT_Float32)
out_ds.SetProjection(ds1.GetProjection())
out_ds.SetGeoTransform(ds1.GetGeoTransform())
band = out_ds.GetRasterBand(1)
band.WriteArray(anomaly)
band.FlushCache()
band.ComputeStatistics(False)
band=None           #important,otherwise will return blank values
out_ds=None